In [1]:
import json
import anthropic
import requests
import openai
import tiktoken

from elasticsearch import Elasticsearch
from tqdm.auto import tqdm


In [2]:
CONFIG = "config.json"
# Read the JSON file to read api key
with open(CONFIG, 'r') as file:
    data = json.load(file)

api_key = data["api-key"]

In [3]:
client = anthropic.Anthropic(api_key=api_key)

In [4]:
def llm(prompt):
    response = client.messages.create(
        # model="claude-3-opus-20240229",
        # model="claude-3-haiku-20240307",
        model="claude-3-sonnet-20240229",
        max_tokens=1024,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.content[0].text

### Homework tasks

In [5]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [6]:
query_hw = "How do I execute a command in a running docker container?"

In [7]:
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': '703e4181e920', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'XFUwYlSGRmuDbmIJ3mjrsg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [8]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "hw_course-questions"

es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'hw_course-questions'})

In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

#### Question 3

In [10]:
def elastic_search_hw(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit)
    
    return result_docs

In [11]:
search_results = elastic_search_hw(query_hw)

print(f"The Top SCORE: {search_results[0].get('_score')}")

The Top SCORE: 83.480255


In [12]:
# Scores for all 5 answers
for item in search_results:
    score = item.get('_score')
    source = item.get('_source')
    if (score is not None) and (source is not None):
        print(f"SOURCE: {source} \nSCORE: {score}")

SOURCE: {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'} 
SCORE: 83.480255
SOURCE: {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServe

#### Question 4

In [13]:
def elastic_search_hw(query):

    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit)
    
    return result_docs

In [14]:
search_results = elastic_search_hw(query_hw)
    
print(f"3rd QUESTION: {search_results[2].get('_source')['question']}")

3rd QUESTION: How do I copy files from a different folder into docker container’s working directory?


In [15]:
# All 3 questions, scores and sources
for item in search_results:
    score = item.get('_score')
    source = item.get('_source')
    question = source['question']
    print(f"QUESTION: {question} \nSOURCE: {source} \nSCORE: {score}")

QUESTION: How do I debug a docker container? 
SOURCE: {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'} 
SCORE: 83.480255
QUESTION: How do I copy files from my local machine to docker container? 
SOURCE: {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kuma

#### Question 5

In [16]:
def build_prompt_hw(query, search_results):
    
    context_template = """
    Q: {question}
    A: {text}
    """.strip()

    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()
    
    context = ""
    
    for doc in search_results:
        context = context + context_template.format(question=doc['_source']['question'], text=doc['_source']['text']).strip() + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [17]:
resulting_prompt = build_prompt_hw(query_hw, search_results)
print(f"The length of resulting prompt is: {len(resulting_prompt)}")

The length of resulting prompt is: 1490


In [18]:
# Print the resulting prompt
print(resulting_prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: How do I execute a command in a running docker container?

    CONTEXT:
    Q: How do I debug a docker container?
    A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
    A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or

#### Question 6

In [19]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [20]:
tokens = encoding.encode(resulting_prompt)
print(f"The length of list of tokens is: {len(tokens)}\n")

The length of list of tokens is: 329



In [21]:
print(f"The length of decoded resulting prompt is: {len(encoding.decode(tokens))}\n")
print("Decoded prompt below:\n")
print(encoding.decode(tokens))


The length of decoded resulting prompt is: 1490

Decoded prompt below:

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: How do I execute a command in a running docker container?

    CONTEXT:
    Q: How do I debug a docker container?
    A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
    A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp co

#### Bonus: generating the answer (ungraded)

In [22]:
def elastic_rag_hw(query):
    search_results = elastic_search_hw(query)
    prompt = build_prompt_hw(query, search_results)
    answer = llm(prompt)
    return answer

In [23]:
response = elastic_rag_hw(query_hw)

In [24]:
print(f"The length of response as a string: {len(response)}\n")

response_tokens = encoding.encode(response)
print(f"The length of list of tokens is: {len(response_tokens)}\n")

print("Response below:\n")
print(response)

The length of response as a string: 1130

The length of list of tokens is: 262

Response below:

To execute a command in a running Docker container, you can use the `docker exec` command. Here's the syntax:

```
docker exec -it <container_id_or_name> <command>
```

Where:

- `-it` connects you to the container's terminal interactively (optional, but recommended for running commands like `bash` or `sh`).
- `<container_id_or_name>` is either the container ID or the container name.
- `<command>` is the command you want to execute inside the container (e.g., `bash`, `ls`, `apt-get install ...`, etc.).

For example, to open an interactive Bash shell inside a running container with the ID `abc123`, you would run:

```
docker exec -it abc123 bash
```

This will give you a terminal inside the container where you can execute commands as if you were running them directly on the host machine.

If you want to execute a single command without opening an interactive shell, you can omit the `-it` fla

#### Bonus: calculating the costs (ungraded)

In [26]:
# Calculations for the Homework Task conditions for OpenAI GPT-4o

# number of requests
requests = 1000
# number of tokens
input = 150
output = 250
# the cost per 1000 tokens for OpenAI
input_cost = 0.005
output_cost = 0.015

cost = (input_cost/1000*input + output_cost/1000*output)*requests
print(f"The cost for homework task conditions (1000 requests for GPT-4o) is {cost} US dollars")

# Calculations for the Homework Task for OpenAI GPT-4o for our prompt and response

# number of tokens
our_input = len(tokens)
our_output = len(response_tokens)

# the cost per 1000 tokens for OpenAI
input_cost = 0.005
output_cost = 0.015

our_cost = (input_cost/1000*our_input + output_cost/1000*our_output)
print(f"The cost of our request for GPT-4o is {round(our_cost, 7)} US dollars")

# Calculations for the Homework Task for Anthropic Claude 3.5 Sonnet for our prompt

# on June 28, the cost per 1000 tokens for Anthropic Claude 3.5 Sonnet (https://www.anthropic.com/pricing#anthropic-api)
input_cost = 0.003
output_cost = 0.015

our_cost = (input_cost/1000*our_input + output_cost/1000*our_output)
print(f"The cost of our request for Claude 3.5 Sonnet is {round(our_cost, 7)} US dollars")

The cost for homework task conditions (1000 requests for GPT-4o) is 4.5 US dollars
The cost of our request for GPT-4o is 0.005575 US dollars
The cost of our request for Claude 3.5 Sonnet is 0.004917 US dollars
